# 🎨 AI Toolkit Web UI on Colab

Run the full ai-toolkit training & generation web UI on **Colab's GPU** with public internet access.

**Time to start**: ~5 minutes

## 🟢 STEP 0: Mount Google Drive (for VAE + Datasets)

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
print("✅ Google Drive mounted at /content/drive")

In [ ]:
# Configure your paths from Google Drive

# 🔴 EDIT THESE with your Google Drive paths
VAE_PATH = "/content/drive/My Drive/checkpoint-7100"  # Path to transparent VAE
DATASET_PATHS = [
    "/content/drive/My Drive/dataset1",
    "/content/drive/My Drive/dataset2",
    "/content/drive/My Drive/dataset3",
]

# Verify they exist
print("🔍 Checking paths...\n")

if os.path.exists(VAE_PATH):
    print(f"✅ VAE found at:")
    print(f"   {VAE_PATH}")
else:
    print(f"❌ VAE not found at:")
    print(f"   {VAE_PATH}")
    print(f"   Make sure you uploaded checkpoint-7100 to Google Drive!")

print()
for i, path in enumerate(DATASET_PATHS, 1):
    if os.path.exists(path):
        files = len(os.listdir(path))
        print(f"✅ Dataset {i}: {files} files")
        print(f"   {path}")
    else:
        print(f"❌ Dataset {i} not found")
        print(f"   {path}")
    print()

## 1️⃣ Check GPU

In [ ]:
import torch

print("🖥️  GPU Info:")
if torch.cuda.is_available():
    print(f"   Device: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("   ✅ GPU available!")
else:
    print("   ⚠️  WARNING: No GPU! Runtime > Change runtime type > GPU")

## 2️⃣ Clone Repository

⚠️ **Replace `YOUR_USERNAME`** with your GitHub username first!

In [ ]:
import os
import subprocess

# 🔴 EDIT THIS LINE - Replace YOUR_USERNAME
repo_url = "https://github.com/YOUR_USERNAME/ai-toolkit-rgba.git"

repo_path = "/content/ai-toolkit"

if not os.path.exists(repo_path):
    print(f"📥 Cloning {repo_url}...")
    subprocess.run(["git", "clone", repo_url, repo_path], check=True)
    print("✅ Clone complete!")
else:
    print("✅ Already cloned")

os.chdir(repo_path)

## 3️⃣ Install Dependencies

⏱️ This takes ~3-5 minutes

In [ ]:
import subprocess
import sys

print("📦 Installing dependencies...")

# Node.js
result = subprocess.run(["node", "--version"], capture_output=True)
if result.returncode != 0:
    print("   Installing Node.js...")
    subprocess.run(["apt-get", "update", "-qq"], capture_output=True)
    subprocess.run(["apt-get", "install", "-y", "-qq", "nodejs", "npm"], check=True)
print("   ✅ Node.js")

# npm packages
print("   Installing npm packages...")
subprocess.run(["npm", "install", "--prefix", "ui"], capture_output=True, check=False)
print("   ✅ npm")

# localtunnel
print("   Installing localtunnel...")
subprocess.run(["npm", "install", "-g", "localtunnel"], capture_output=True, check=False)
print("   ✅ localtunnel")

print("\n✅ All dependencies installed!")

## 4️⃣ Build UI

⏱️ This takes ~2-3 minutes

In [ ]:
import subprocess

print("🔨 Building Next.js UI...")
result = subprocess.run(
    ["npm", "run", "build", "--prefix", "ui"],
    capture_output=True,
    text=True,
    timeout=300
)

if result.returncode == 0:
    print("✅ Build successful!")
else:
    print("⚠️  Build completed with warnings (usually OK)")

## 5️⃣ Start Web UI

🚀 This creates a public URL you can access from anywhere!

**Keep this cell running!** The public URL will appear below.

In [ ]:
import subprocess
import time
import os

os.chdir("/content/ai-toolkit")
UI_PORT = 8675

print("🚀 Starting web UI server...")
ui_process = subprocess.Popen(
    ["npm", "run", "start", "--prefix", "ui"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print("⏳ Waiting 15 seconds for server to start...")
time.sleep(15)

print(f"\n🌐 Creating tunnel on port {UI_PORT}...\n")
tunnel_process = subprocess.Popen(
    ["lt", "--port", str(UI_PORT), "--local-host", "127.0.0.1"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# Read tunnel output
url_found = False
for line in tunnel_process.stdout:
    line = line.strip()
    if line:
        print(line)
    
    if ".loca.lt" in line and not url_found:
        url_found = True
        print("\n" + "="*70)
        print("✅ WEB UI IS LIVE!")
        print("="*70)
        print(f"\n🔗 PUBLIC URL:\n\n   {line}\n")
        print("="*70)
        print("\n📝 What you can do:")
        print("   • Open the URL in your browser")
        print("   • Create training jobs (VAE + datasets auto-detected)")
        print("   • Generate images with LoRA")
        print("   • Monitor GPU/VRAM")
        print("\n⏱️  Keep this cell running!")
        print("   You can open NEW cells to run other commands.\n")

# Keep alive
try:
    tunnel_process.wait()
except KeyboardInterrupt:
    print("\n⏹️  Shutting down...")
    tunnel_process.terminate()
    ui_process.terminate()

## 🛠️ Helper Commands

Run these in separate cells while the server is running:

In [ ]:
# Check GPU usage
import subprocess
result = subprocess.run(["nvidia-smi"], capture_output=True, text=True)
print(result.stdout)

In [ ]:
# Check training outputs
import os
from pathlib import Path

output_dir = Path("/content/ai-toolkit/output")
if output_dir.exists():
    print("📁 Training outputs:")
    for folder in sorted(output_dir.iterdir()):
        if folder.is_dir():
            print(f"\n   📂 {folder.name}")
            samples = list(folder.glob("*.png"))
            if samples:
                print(f"      Files: {len(samples)}")
else:
    print("No outputs yet")